In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from difflib import SequenceMatcher

In [2]:
df = pd.read_csv('parkingLot.csv')
df.head()

,vehicle_no,timestamp,camera_id
0,MHUN7063,2024-09-12 05:00:00,1
1,MHYN4677,2024-09-12 05:00:00,1
2,MHEL6595,2024-09-12 05:00:00,1
3,MHNQ2590,2024-09-12 05:00:00,1
4,MHHA0518,2024-09-12 05:00:00,1


In [3]:
#cnvert vehicle_np to string
df['vehicle_no'] = df['vehicle_no'].astype(str)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [27]:
df

,vehicle_no,timestamp,camera_id
0,MHUN7063,2024-09-12 05:00:00,1
1,MHYN4677,2024-09-12 05:00:00,1
2,MHEL6595,2024-09-12 05:00:00,1
3,MHNQ2590,2024-09-12 05:00:00,1
4,MHHA0518,2024-09-12 05:00:00,1
...,...,...,...
106689,MHCN7114,2024-11-14 00:00:00,2
106690,MHVY8418,2024-11-14 00:00:00,2
106691,MHSR8102,2024-11-14 00:00:00,2
106692,MHQG5264,2024-11-14 00:00:00,2


In [19]:
# drop empty string in vehicle_no

no_na_data = df[df['vehicle_no'] != 'nan']
entries = no_na_data[no_na_data['camera_id']==1]
exits = no_na_data[no_na_data['camera_id']==2]
no_na_data

,vehicle_no,timestamp,camera_id
0,MHUN7063,2024-09-12 05:00:00,1
1,MHYN4677,2024-09-12 05:00:00,1
2,MHEL6595,2024-09-12 05:00:00,1
3,MHNQ2590,2024-09-12 05:00:00,1
4,MHHA0518,2024-09-12 05:00:00,1
...,...,...,...
106689,MHCN7114,2024-11-14 00:00:00,2
106690,MHVY8418,2024-11-14 00:00:00,2
106691,MHSR8102,2024-11-14 00:00:00,2
106692,MHQG5264,2024-11-14 00:00:00,2


In [20]:
paired_data = pd.merge(entries, exits, on='vehicle_no', suffixes=('_entry', '_exit'))
paired_data = paired_data[paired_data['timestamp_entry'] < paired_data['timestamp_exit']]
paired_data

,vehicle_no,timestamp_entry,camera_id_entry,timestamp_exit,camera_id_exit
0,MHUN7063,2024-09-12 05:00:00,1,2024-09-12 08:00:00,2
1,MHYN4677,2024-09-12 05:00:00,1,2024-09-12 08:00:00,2
2,MHEL6595,2024-09-12 05:00:00,1,2024-09-12 08:00:00,2
3,MHNQ2590,2024-09-12 05:00:00,1,2024-09-12 08:20:00,2
4,MHGS9163,2024-09-12 05:00:00,1,2024-09-12 08:00:00,2
...,...,...,...,...,...
52755,MHVY8418,2024-11-13 20:00:00,1,2024-11-14 00:00:00,2
52756,MHDF3718,2024-11-13 20:00:00,1,2024-11-14 00:00:00,2
52757,MHQJ4009,2024-11-13 20:00:00,1,2024-11-14 00:00:00,2
52758,MHPN3450,2024-11-13 20:00:00,1,2024-11-14 00:00:00,2


In [22]:
entries = df[df['camera_id']==1]
exits = df[df['camera_id']==2]

In [23]:
unmatched_entries = entries[~entries['vehicle_no'].isin(paired_data['vehicle_no'])]
unmatched_exits = exits[~exits['vehicle_no'].isin(paired_data['vehicle_no'])]

In [24]:
unmatched_entries['date'] = unmatched_entries['timestamp'].dt.date
unmatched_exits['date'] = unmatched_exits['timestamp'].dt.date

C:\Users\vaibh\AppData\Local\Temp\ipykernel_14976\3890901745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_entries['date'] = unmatched_entries['timestamp'].dt.date
C:\Users\vaibh\AppData\Local\Temp\ipykernel_14976\3890901745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_exits['date'] = unmatched_exits['timestamp'].dt.date


In [25]:
orphan_pairs = []
for entry_idx, entry_row in unmatched_entries.iterrows():
    entry_date = entry_row['timestamp'].date()
    same_day_exits = unmatched_exits[unmatched_exits['timestamp'].dt.date == entry_date]

    if not same_day_exits.empty:
        
        similarities = same_day_exits['vehicle_no'].apply(lambda x: SequenceMatcher(None, entry_row['vehicle_no'], x).ratio())
        most_similar_exit_idx = similarities.idxmax()  
        
        if similarities[most_similar_exit_idx] > 0.1:
            exit_row = same_day_exits.loc[most_similar_exit_idx]
            if entry_row['timestamp'] < exit_row['timestamp']:  
                orphan_pairs.append({
                    'vehicle_no': entry_row['vehicle_no'],
                    'timestamp_entry': entry_row['timestamp'],
                    'timestamp_exit': exit_row['timestamp']
                })
            unmatched_exits = unmatched_exits.drop(most_similar_exit_idx)  


orphan_pairs_df = pd.DataFrame(orphan_pairs)
orphan_pairs_df

,vehicle_no,timestamp_entry,timestamp_exit
0,MHHA0518,2024-09-12 05:00:00,2024-09-12 08:40:00
1,MHFT7338,2024-09-12 06:00:00,2024-09-12 10:40:00
2,MHPK8268,2024-09-12 09:00:00,2024-09-12 14:00:00
3,MHWY3805,2024-09-12 11:00:00,2024-09-12 14:20:00
4,MHIH5246,2024-09-12 11:40:00,2024-09-12 15:20:00
...,...,...,...
591,MHNF6986,2024-11-13 10:00:00,2024-11-13 14:40:00
592,MHIO9771,2024-11-13 10:20:00,2024-11-13 16:00:00
593,MHZS7809,2024-11-13 10:40:00,2024-11-13 17:00:00
594,MHLE5669,2024-11-13 17:00:00,2024-11-13 22:40:00


In [26]:
final_data = pd.concat([paired_data[['vehicle_no', 'timestamp_entry', 'timestamp_exit']], orphan_pairs_df], ignore_index=True)
final_data = final_data.sort_values(by='timestamp_entry')
final_data

,vehicle_no,timestamp_entry,timestamp_exit
0,MHUN7063,2024-09-12 05:00:00,2024-09-12 08:00:00
20,MHMY1867,2024-09-12 05:00:00,2024-09-12 09:20:00
19,MHZF8417,2024-09-12 05:00:00,2024-09-12 09:20:00
18,MHFU6425,2024-09-12 05:00:00,2024-09-12 09:40:00
17,MHHB4038,2024-09-12 05:00:00,2024-09-12 09:20:00
...,...,...,...
52574,MHDF3718,2024-11-13 20:00:00,2024-11-14 00:00:00
52575,MHQJ4009,2024-11-13 20:00:00,2024-11-14 00:00:00
52576,MHPN3450,2024-11-13 20:00:00,2024-11-14 00:00:00
52577,MHBB9719,2024-11-13 20:00:00,2024-11-14 00:00:00


In [28]:
# save the csv
final_data.to_csv('parking_data_entry_exit.csv', index=False)